In [15]:
import os
from subprocess import check_output
import shlex
os.chdir(check_output(shlex.split("git rev-parse --show-toplevel")).strip().decode('ascii'))

In [16]:
import pickle as pkl
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual

import scipy
from scipy.stats import spearmanr

import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt
from VTK.VTKMesh import VTKObject as Mesh

In [17]:
wall_thickness = pkl.load(open("data/transforms/cached/wall_thickness.pkl", "rb"))

In [18]:
wall_thickness["wall_thickness_epicardial"].shape

(32647, 1797)

In [19]:
normalized_wt = []
for i, wt in enumerate(wall_thickness["wall_thickness_epicardial"]):
    cbrt_vol = wall_thickness["convex_hull_volumes"][i]**(1./3)
    normalized_wt.append(wt/cbrt_vol)
normalized_wt = np.array(normalized_wt)

In [20]:
output_dir = "output"
experiments = [x for x in os.listdir(output_dir) if os.path.exists(os.path.join(output_dir, x, ".finished"))]

w = widgets.Dropdown(
    value="2020-09-11_02-13-41",
    options=experiments,
    description='Experiment:',
    disabled=False,
)

display(w)

Dropdown(description='Experiment:', index=499, options=('2020-10-12_03-44-42', '2020-10-12_03-07-22', '2020-10…

In [22]:
import ExperimentClass
experiment = ExperimentClass.ComaExperiment(os.path.join(output_dir, w.value))
experiment.load_model()

2021-02-15 22:21:08,246 [INFO] No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
2021-02-15 22:21:08,740 [ERROR] Unsupported point attribute type: point_data for file: ./template/template.vtk


In [26]:
z = experiment.load_z()

# reorder
ids = np.array([int(x) for x in wall_thickness["ids"]])
# z = z.set_index("ID").loc[ids,:].drop("subset", 1)
# z = np.array(z) # Pandas DataFrame to Numpy array

In [27]:
dic = pkl.load(open("data/transforms/cached/2ch_segmentation__LV__ED__non_scaled__dic.pkl", "rb"))

In [29]:
# Get indices of each subpartition (LV endo and epi)
LVRV = Mesh("template/template.vtk").extractSubpart([1,2])

# Booleans indicating vertices that belong to each surface
endo_j = (LVRV.subpartID == 1)
epi_j = (LVRV.subpartID == 2)

2021-02-15 22:22:50,234 [ERROR] Unsupported point attribute type: point_data for file: template/template.vtk


In [30]:
faces = np.hstack([[3] + list(x) for x in LVRV.triangles])
LVRV_pv = pv.PolyData(dic["mean"], faces)

In [35]:
id = 2
# prepending a 3 before each triangle (PyVista format for faces)
faces_epi = np.hstack([[3] + list(x) for x in lv_epi.triangles])
lv_epi_pv = pv.PolyData(LVRV.points[epi_j], faces_epi)

faces_endo = np.hstack([[3] + list(x) for x in lv_endo.triangles])
lv_endo_pv = pv.PolyData(endo[id], faces_endo)

# lv_epi_pv.plot(scalars=wall_thickness[id])
# lv_epi_pv.rotate_z(180)
# lv_endo_pv.rotate_z(180)

NameError: name 'lv_epi' is not defined

In [33]:
corr = spearmanr(z, normalized_wt, axis=0)

In [26]:
def plot_mesh(mesh, faces, angle=0):
    
  surf = pv.PolyData(mesh, faces)
    
  surf.rotate_z(angle)
  plotter = pv.Plotter(notebook=True)
  # kargs = {"point_size": 2, "render_points_as_spheres": True}
    
  # surf.plot() #, **kargs)
  plotter.add_mesh(surf, show_edges=True)
  plotter.show(interactive=True)

  plotter.enable()

In [39]:
def f(id, angle):
  
  lv_epi_pv = pv.PolyData(epi[id], faces_epi)
  lv_endo_pv = pv.PolyData(endo[id], faces_endo)

  lv_epi_pv.rotate_z(angle)
  lv_endo_pv.rotate_z(angle)
  
  plotter = pv.Plotter(notebook=False)
          
  # plotter.add_mesh(lv_epi_pv, opacity=0.8, scalars=wall_thickness['wall_thickness_epicardial'][id])
  plotter.add_mesh(lv_epi_pv, opacity=1, scalars=corr.correlation[8:,(1,)])
  plotter.add_mesh(lv_endo_pv, opacity=1)    
  plotter.show(interactive=True)
  plotter.enable()

interact(f, 
  id = widgets.SelectionSlider(options=range(200)),
  angle = widgets.SelectionSlider(options=range(360))
)

interactive(children=(SelectionSlider(description='id', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,…

<function __main__.f(id, angle)>

In [34]:
def f():    
  plotter = pv.Plotter(notebook=True)          
  # plotter.add_mesh(LVRV_pv, opacity=0.8)#, scalars=wall_thickness['wall_thickness_epicardial'][id])
  plotter.add_mesh(lv_epi_pv, opacity=1, scalars=corr.correlation[8:,(1,)])
  plotter.add_mesh(lv_endo_pv, opacity=1)    
  plotter.show(interactive=True)
  plotter.enable()

f()

NameError: name 'lv_endo_pv' is not defined